In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity


In [24]:
df_choice = pd.read_csv('choice_data.csv',sep=',',index_col=0)

df_choice['part_control'] = (df_choice['group_value'] == False)*df_choice['part_value']
df_choice['part_treatment'] = (df_choice['group_value'] == True)*df_choice['part_value']
 
sum_choice_seq = df_choice.groupby('worker_id')['choice_value'].sum().to_frame()
analysis_workers = sum_choice_seq[(sum_choice_seq['choice_value'] != 24) & (sum_choice_seq['choice_value'] != 0)].index
# df_analysis = df_choice[df_choice['worker_id'].isin(analysis_workers)]
df = df_choice

bool_cols = df.select_dtypes(include=['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)

In [3]:
tab_reward = (df.groupby(['front_ratio','single_amount'])['choice_value'].count()/ len(df)).reset_index()
tab_reward = tab_reward.rename(columns={'choice_value':'prob'})

var = {'front_ratio':0.1,'single_amount':200}

def kernelReward(var,tab=None):
    
    reward = None
    for key, value in var.items():
        if reward is None:
            reward = tab[key] == value
        else:
            reward &= tab[key] == value

    # Find corresponding row
    _row = tab[reward]

    return _row['prob'].values[0]

In [4]:
kernelReward(var,tab=tab_reward)

0.041666666666666664

In [7]:
logitFunc = lambda s: np.exp(s)/(1+np.exp(s))
logitPrime = lambda s: np.exp(-s)/(1+np.exp(-s))**2
logLink = lambda y: np.log(y/(1-y))
V = lambda m: m*(1-m)
G = lambda s: logitFunc(s)*V(logitPrime(s)) 



In [29]:
y = df['choice_value']
X = df[['group_value','part_value','part_control','part_treatment']]
X['const'] = 1
X

C:\Users\zarkwang\AppData\Local\Temp\ipykernel_15556\4118028202.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['const'] = 1


,group_value,part_value,part_control,part_treatment,const
13,1,1,0,1,1
14,1,1,0,1,1
15,1,1,0,1,1
16,1,1,0,1,1
17,1,1,0,1,1
...,...,...,...,...,...
3895,0,0,0,0,1
3896,0,0,0,0,1
3897,0,0,0,0,1
3898,0,0,0,0,1


In [30]:
adjust_y = 1e-5


logLink(2* (0.5-y) * adjust_y + y) - 

13      11.512915
14      11.512915
15      11.512915
16      11.512915
17      11.512915
          ...    
3895   -11.512915
3896   -11.512915
3897   -11.512915
3898   -11.512915
3899   -11.512915
Name: choice_value, Length: 7056, dtype: float64

In [ ]:

# Generate some random data
np.random.seed(0)
data = np.random.randn(1000)

# Instantiate and fit the KDE model
kde = KernelDensity(bandwidth=0.5, kernel='gaussian')
kde.fit(data[:, None])

# Generate points for evaluation
x = np.linspace(-5, 5, 1000)
log_density = kde.score_samples(x[:, None])

# Plot the KDE
plt.fill_between(x, np.exp(log_density), alpha=0.5)
plt.xlabel('x')
plt.ylabel('Density')
plt.title('Kernel Density Estimation')
plt.show()